In [5]:
import pandas as pd
import numpy as np
import polars as pl
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import gzip
import shutil
import pathlib
import os
import sqlalchemy
import sqlite3
import spacy
import re
import tqdm
from tqdm.notebook import tqdm, trange
import ipywidgets as widgets
from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import time
import timeit
import sklearn

from transformers import BertTokenizer, BertForSequenceClassification
import torch
from torch.nn.functional import softmax

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()

%matplotlib inline
alt.data_transformers.disable_max_rows()

nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load('/path/to/en_core_web_sm')

In [6]:
def print_files_in_directory(directory_path):
    with os.scandir(directory_path) as entries:
        for entry in entries:
            if entry.is_file():
                print(entry.name)

def get_sentiment_score(text):
    sentiment_scores = sid_obj.polarity_scores(text)
    return sentiment_scores

def clean_text(text):
    doc = nlp(text)

    cleaned_tokens = [token.lemma_.lower().strip() for token in doc if not token.is_punct and not token.is_space]

    cleaned_tokens = [token for token in cleaned_tokens if not nlp.vocab[token].is_stop]

    cleaned_text = ' '.join(cleaned_tokens)

    return cleaned_text

In [7]:
conn = sqlite3.connect(r"C:/Users/asl4a/AirBnB_Data.db")
cursor = conn.cursor()

In [8]:
sql = """
SELECT reviews.id, reviews.temp_index, reviews.comments
FROM reviews"""

df_revs = pd.read_sql(sql,con=conn)

In [9]:
df_revs.head()

,id,temp_index,comments
0,1927,0,I can't say enough about how wonderful it was ...
1,3867,1,Michelle and Collier's home is wonderful! They...
2,4159,2,I spent one night at Michele's home and felt j...
3,5724,3,Michele and Collier are two of the loveliest p...
4,11891,4,We had the most lovely time staying with Miche...


In [10]:
df_revs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668045 entries, 0 to 9668044
Data columns (total 3 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          int64 
 1   temp_index  int64 
 2   comments    object
dtypes: int64(2), object(1)
memory usage: 221.3+ MB


In [11]:
df_revs['comments'] = df_revs['comments'].astype(str)